In [1]:
import numpy as np
import time
import generalized_genSim_tel_aviv as ggsd
import curve_fitting_tel_aviv as cf
import matplotlib.pyplot as plt
from scipy import optimize, stats
import bluepyopt as bpop
import bluepyopt.deapext.algorithms as algo
import vclamp_evaluator_relative as vcl_ev
import pickle
from deap import tools
#from deap import base, creator
import multiprocessing


In [2]:
evaluator = vcl_ev.Vclamp_evaluator_relative('./param_stats.csv', 'na16')

C:\Users\ccfla\desktop\simulating\Tel_Aviv_folder\generalized_genSim_tel_aviv.py:693: RuntimeWarning: invalid value encountered in log
  tau = -h.tstop / np.log(-RF_t + 1)
C:\Users\ccfla\desktop\simulating\Tel_Aviv_folder\generalized_genSim_tel_aviv.py:693: RuntimeWarning: invalid value encountered in log
  tau = -h.tstop / np.log(-RF_t + 1)
C:\Users\ccfla\desktop\simulating\Tel_Aviv_folder\curve_fitting_tel_aviv.py:25: RuntimeWarning: overflow encountered in exp
  return y0 + (plateau - y0) * (1 - np.exp(-k * x))
C:\Users\ccfla\desktop\simulating\Tel_Aviv_folder\curve_fitting_tel_aviv.py:25: RuntimeWarning: overflow encountered in exp
  return y0 + (plateau - y0) * (1 - np.exp(-k * x))


In [3]:
evaluator.wild_data

{'v_half_act': -37.955531038518814,
 'gv_slope': 5.272052475529,
 'v_half_ssi': -70.21581681391069,
 'ssi_slope': 9.754098918435815,
 'tau': 7169.530544749319,
 'persistent10': 0,
 'persistent20': 0}

{'v_half_act': -37.955531038518814,
 'gv_slope': 5.272052475529,
 'v_half_ssi': -70.21581681391069,
 'ssi_slope': 9.754098918435815,
 'tau': 7169.530544749319,
 'persistent10': 0,
 'persistent20': 0}

In [4]:
gen_counter = 0
best_indvs = []
cp_freq = 1
old_update = algo._update_history_and_hof
def my_update(halloffame, history, population):
    global gen_counter,cp_freq
    if halloffame is not None:
        halloffame.update(population)
    
    if halloffame:
        best_indvs.append(halloffame[0])
    gen_counter = gen_counter+1
    print("Current generation: ", gen_counter)
    if gen_counter%cp_freq == 0:
        fn = '.pkl'
        save_logs(fn,best_indvs,population)

def my_record_stats(stats, logbook, gen, population, invalid_count):
    '''Update the statistics with the new population'''
    record = stats.compile(population) if stats is not None else {}
    logbook.record(gen=gen, nevals=invalid_count, **record)
    print('log: \n', logbook, '\n')
    output = open("log.pkl", 'wb')
    pickle.dump(logbook, output)
    output.close()

def save_logs(fn, best_indvs, hof):
    output = open("indv"+fn, 'wb')
    pickle.dump(best_indvs, output)
    output.close()
    output = open("hof"+fn, 'wb')
    pickle.dump(hof, output)


In [5]:
#hof = tools.HallOfFame(1, similar=np.array_equal)
hof = tools.ParetoFront()
algo._update_history_and_hof = my_update
algo._record_stats = my_record_stats
pool = multiprocessing.Pool(processes=64)
deap_opt = bpop.optimisations.DEAPOptimisation(evaluator, offspring_size=2, hof = hof, map_function=pool.map)
#deap_opt = bpop.optimisations.DEAPOptimisation(evaluator, offspring_size=5, hof = hof)
cp_file = './cp.pkl'

Exception in thread Thread-9:
Traceback (most recent call last):
  File "C:\Users\ccfla\AppData\Local\Programs\Python\Python39\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "C:\Users\ccfla\AppData\Local\Programs\Python\Python39\lib\threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ccfla\AppData\Local\Programs\Python\Python39\lib\multiprocessing\pool.py", line 519, in _handle_workers
    cls._wait_for_updates(current_sentinels, change_notifier)
  File "C:\Users\ccfla\AppData\Local\Programs\Python\Python39\lib\multiprocessing\pool.py", line 499, in _wait_for_updates
    wait(sentinels, timeout=timeout)
  File "C:\Users\ccfla\AppData\Local\Programs\Python\Python39\lib\multiprocessing\connection.py", line 884, in wait
    ready_handles = _exhaustive_wait(waithandle_to_obj.keys(), timeout)
  File "C:\Users\ccfla\AppData\Local\Programs\Python\Python39\lib\multiprocessing\connection.py", line 816, in _exhaustive_wait
    re

In [6]:
start_time = time.time()
pop, hof, log, hst = deap_opt.run(max_ngen=2, cp_filename=cp_file)
#pop, hof, log, hst = deap_opt.run(max_ngen=10, cp_filename=None)
end_time = time.time()
print(end_time - start_time)

AssertionError: Assigned values have not the same length than fitness weights

AssertionError: Assigned values have not the same length than fitness weights

In [ ]:
print(log)

In [ ]:
evaluator.plot_data(best_indvs[-1], 'nax')

In [ ]:
 y0, plateau, k, tau = cf.calc_recov_obj('nax')

In [ ]:
best_indvs

In [ ]:
for i in range(1, 26):
    print('Generation: ' + str(i))
    evaluator.plot_data(best_indvs[i - 1], 'nax')

In [ ]:
x = [[329.05329639102604, 0.1606308091165473, 0.03747348501333136, 0.06526885803219298, 190.9036832318502, 0.1796721956330009, 3.1289379891044624, 0.3334967529455608, 612.3972657411969, 0.3025076706861414, 3.025676019511704, 0.1259582839920209, 7.510910096536974, 0.09873672648427731, 0.45240175567275553, 2.432778951401892, 936.167509954995, 0.0192159423779658, 7.098868873725916, 2.7656822791019575], [49.14258693986015, 0.06379311269967337, 0.12928835934449193, 0.35728356783756815, 178.51913788169645, 0.12218913898530714, 1.4330430326857246, 0.010630088530980991, 451.0203901597099, 0.35791607365034656, 2.7922869417294307, 0.12569068714912426, 19.70501066739743, 2.1216048388520674, 0.17091060192878565, 1.8316537498896295, 788.2120783433784, 0.06239596387525991, 7.165917086754051, 11.427466036120485], [49.14258693986015, 0.06379311269967337, 0.12928835934449193, 0.35728356783756815, 178.51913788169645, 0.12218913898530714, 1.4330430326857246, 0.010630088530980991, 451.0203901597099, 0.35791607365034656, 2.7922869417294307, 0.12569068714912426, 19.70501066739743, 2.1216048388520674, 0.17091060192878565, 1.8316537498896295, 788.2120783433784, 0.06239596387525991, 7.165917086754051, 11.427466036120485], [49.14258693986015, 0.06379311269967337, 0.12928835934449193, 0.35728356783756815, 178.51913788169645, 0.12218913898530714, 1.4330430326857246, 0.010630088530980991, 451.0203901597099, 0.35791607365034656, 2.7922869417294307, 0.12569068714912426, 19.70501066739743, 2.1216048388520674, 0.17091060192878565, 1.8316537498896295, 788.2120783433784, 0.06239596387525991, 7.165917086754051, 11.427466036120485], [49.14258693986015, 0.06379311269967337, 0.12928835934449193, 0.35728356783756815, 178.51913788169645, 0.12218913898530714, 1.4330430326857246, 0.010630088530980991, 451.0203901597099, 0.35791607365034656, 2.7922869417294307, 0.12569068714912426, 19.70501066739743, 2.1216048388520674, 0.17091060192878565, 1.8316537498896295, 788.2120783433784, 0.06239596387525991, 7.165917086754051, 11.427466036120485], [49.14258693986015, 0.06379311269967337, 0.12928835934449193, 0.35728356783756815, 178.51913788169645, 0.12218913898530714, 1.4330430326857246, 0.010630088530980991, 451.0203901597099, 0.35791607365034656, 2.7922869417294307, 0.12569068714912426, 19.70501066739743, 2.1216048388520674, 0.17091060192878565, 1.8316537498896295, 788.2120783433784, 0.06239596387525991, 7.165917086754051, 11.427466036120485], [49.14258693986015, 0.06379311269967337, 0.12928835934449193, 0.35728356783756815, 178.51913788169645, 0.12218913898530714, 1.4330430326857246, 0.010630088530980991, 451.0203901597099, 0.35791607365034656, 2.7922869417294307, 0.12569068714912426, 19.70501066739743, 2.1216048388520674, 0.17091060192878565, 1.8316537498896295, 788.2120783433784, 0.06239596387525991, 7.165917086754051, 11.427466036120485], [49.14258693986015, 0.06379311269967337, 0.12928835934449193, 0.35728356783756815, 178.51913788169645, 0.12218913898530714, 1.4330430326857246, 0.010630088530980991, 451.0203901597099, 0.35791607365034656, 2.7922869417294307, 0.12569068714912426, 19.70501066739743, 2.1216048388520674, 0.17091060192878565, 1.8316537498896295, 788.2120783433784, 0.06239596387525991, 7.165917086754051, 11.427466036120485], [49.14258693986015, 0.06379311269967337, 0.12928835934449193, 0.35728356783756815, 178.51913788169645, 0.12218913898530714, 1.4330430326857246, 0.010630088530980991, 451.0203901597099, 0.35791607365034656, 2.7922869417294307, 0.12569068714912426, 19.70501066739743, 2.1216048388520674, 0.17091060192878565, 1.8316537498896295, 788.2120783433784, 0.06239596387525991, 7.165917086754051, 11.427466036120485], [49.14258693986015, 0.06379311269967337, 0.12928835934449193, 0.35728356783756815, 178.51913788169645, 0.12218913898530714, 1.4330430326857246, 0.010630088530980991, 451.0203901597099, 0.35791607365034656, 2.7922869417294307, 0.12569068714912426, 19.70501066739743, 2.1216048388520674, 0.17091060192878565, 1.8316537498896295, 788.2120783433784, 0.06239596387525991, 7.165917086754051, 11.427466036120485], [49.14258693986015, 0.06379311269967337, 0.12928835934449193, 0.35728356783756815, 178.51913788169645, 0.12218913898530714, 1.4330430326857246, 0.010630088530980991, 451.0203901597099, 0.35791607365034656, 2.7922869417294307, 0.12569068714912426, 19.70501066739743, 2.1216048388520674, 0.17091060192878565, 1.8316537498896295, 788.2120783433784, 0.06239596387525991, 7.165917086754051, 11.427466036120485], [49.14258693986015, 0.06379311269967337, 0.12928835934449193, 0.35728356783756815, 178.51913788169645, 0.12218913898530714, 1.4330430326857246, 0.010630088530980991, 451.0203901597099, 0.35791607365034656, 2.7922869417294307, 0.12569068714912426, 19.70501066739743, 2.1216048388520674, 0.17091060192878565, 1.8316537498896295, 788.2120783433784, 0.06239596387525991, 7.165917086754051, 11.427466036120485], [49.14258693986015, 0.06379311269967337, 0.12928835934449193, 0.35728356783756815, 178.51913788169645, 0.12218913898530714, 1.4330430326857246, 0.010630088530980991, 451.0203901597099, 0.35791607365034656, 2.7922869417294307, 0.12569068714912426, 19.70501066739743, 2.1216048388520674, 0.17091060192878565, 1.8316537498896295, 788.2120783433784, 0.06239596387525991, 7.165917086754051, 11.427466036120485], [49.14258693986015, 0.06379311269967337, 0.12928835934449193, 0.35728356783756815, 178.51913788169645, 0.12218913898530714, 1.4330430326857246, 0.010630088530980991, 451.0203901597099, 0.35791607365034656, 2.7922869417294307, 0.12569068714912426, 19.70501066739743, 2.1216048388520674, 0.17091060192878565, 1.8316537498896295, 788.2120783433784, 0.06239596387525991, 7.165917086754051, 11.427466036120485], [49.14258693986015, 0.06379311269967337, 0.12928835934449193, 0.35728356783756815, 178.51913788169645, 0.12218913898530714, 1.4330430326857246, 0.010630088530980991, 451.0203901597099, 0.35791607365034656, 2.7922869417294307, 0.12569068714912426, 19.70501066739743, 2.1216048388520674, 0.17091060192878565, 1.8316537498896295, 788.2120783433784, 0.06239596387525991, 7.165917086754051, 11.427466036120485], [49.14258693986015, 0.06379311269967337, 0.12928835934449193, 0.35728356783756815, 178.51913788169645, 0.12218913898530714, 1.4330430326857246, 0.010630088530980991, 451.0203901597099, 0.35791607365034656, 2.7922869417294307, 0.12569068714912426, 19.70501066739743, 2.1216048388520674, 0.17091060192878565, 1.8316537498896295, 788.2120783433784, 0.06239596387525991, 7.165917086754051, 11.427466036120485], [49.14258693986015, 0.06379311269967337, 0.12928835934449193, 0.35728356783756815, 178.51913788169645, 0.12218913898530714, 1.4330430326857246, 0.010630088530980991, 451.0203901597099, 0.35791607365034656, 2.7922869417294307, 0.12569068714912426, 19.70501066739743, 2.1216048388520674, 0.17091060192878565, 1.8316537498896295, 788.2120783433784, 0.06239596387525991, 7.165917086754051, 11.427466036120485], [49.14258693986015, 0.06379311269967337, 0.12928835934449193, 0.35728356783756815, 178.51913788169645, 0.12218913898530714, 1.4330430326857246, 0.010630088530980991, 451.0203901597099, 0.35791607365034656, 2.7922869417294307, 0.12569068714912426, 19.70501066739743, 2.1216048388520674, 0.17091060192878565, 1.8316537498896295, 788.2120783433784, 0.06239596387525991, 7.165917086754051, 11.427466036120485], [49.14258693986015, 0.06379311269967337, 0.12928835934449193, 0.35728356783756815, 178.51913788169645, 0.12218913898530714, 1.4330430326857246, 0.010630088530980991, 451.0203901597099, 0.35791607365034656, 2.7922869417294307, 0.12569068714912426, 19.70501066739743, 2.1216048388520674, 0.17091060192878565, 1.8316537498896295, 788.2120783433784, 0.06239596387525991, 7.165917086754051, 11.427466036120485], [49.14258693986015, 0.06379311269967337, 0.12928835934449193, 0.35728356783756815, 178.51913788169645, 0.12218913898530714, 1.4330430326857246, 0.010630088530980991, 451.0203901597099, 0.35791607365034656, 2.7922869417294307, 0.12569068714912426, 19.70501066739743, 2.1216048388520674, 0.17091060192878565, 1.8316537498896295, 788.2120783433784, 0.06239596387525991, 7.165917086754051, 11.427466036120485]]

In [ ]:
for i in range(len(x)):
    print(sum(evaluator.evaluate_with_lists(x[i])))

In [ ]:
x[3] == x[len(x) - 1]